In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name and compute calass for your CPU cluster 
cpu_cluster_name = "biaggi-cluster"
desired_vm_size = "Standard_D2_V2" 

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=desired_vm_size,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

zAS""" Q    ` """.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "keep_probability": uniform(0.05, 0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)

# Specify a Policy using median stop i prefer that than a bandit so i wont stop early even if i have to run more
policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = Estimator(source_directory=os.path.join('.', 'scripts'),
                      entry_script='train.py',
                      compute_target=cpu_cluster_name)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
primary_metric_name="accuracy",
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
experiment = Experiment(ws, ws.name)
hyperdrive_run = experiment.submit(hyperdrive_config)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

In [ ]:
from azureml.core.dataset import Dataset
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
subscription_id = '26806ae2-7725-4970-9c73-e6b2c7c706c1'
resource_group = 'aml-quickstarts-122846'
workspace_name = 'quick-starts-ws-122846'

workspace = Workspace(subscription_id, resource_group, workspace_name)
#data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
#dataset = Dataset.Tabular.from_delimited_files(data)
Dataset.get_by_name(workspace, name='bankmarketing-dataset')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(   experiment_timeout_minutes=30,
                                task='classification',
                                primary_metric= 'AUC_weighted',
                                experiment_exit_score = 0.9984,
                                blocked_models = ['KNN','LinearSVM'],
                                enable_onnx_compatible_models=True,
                                training_data= dataset,
                                label_column_name='loan',
                                n_cross_validations= 5)

                            

In [2]:
# Submit your automl run

r_run  = experiment.submit(AutoMLConfig, show_output = False)

In [ ]:
# Retrieve and save your best automl model.
#clearn up the resources
compute_target.delete()
best_run, fitted_model = r_run.get_output()
print(best_run)
print(fitted_model)